# Data Warehouse-Datenbank erstellen


---

Datensatz: [Synthea Breast Cancer Dataset](https://github.com/Fuenfgeld/DMA2023TeamA/tree/main/Daten/Quelldaten)

Primär- und Fremdschlüsseldefinitionen: [Synthea GitHub Repository](https://github.com/synthetichealth/synthea/wiki/CSV-File-Data-Dictionary)

Projektgruppe GitHub Repository: [DMA2023TeamA](https://github.com/Fuenfgeld/DMA2023TeamA)

Source-DB: [GoogleDrive Ablage](https://drive.google.com/drive/folders/1k5cfjGXjNHmwQkydzjTdVHoBvCniBU_W), erstellt mit [Setup_and_fill_Database.ipynb](https://github.com/Fuenfgeld/DMA2023TeamA/blob/main/Code/Setup_and_fill_Database.ipynb)

Mithilfe dieses Colab-Books wird aus der Source-Datenbank (= Staging-DB) eine Data Warehouse-Datenbank (DWH-Datenbank) (= Reporting-DB) erstellt, welche als OLAP-Datenbank für Analysen dient. 



*Version*: 3.0

Version Date: 04/02/2023

Changes: 

*   Versionsnummern der verwendeten Pakete ausgeben (Link: https://colab.research.google.com/drive/1hEBFYWiAFUNKBRinhEY43eOEqXeaHohi#scrollTo=0CiLSmoxK2EH&line=3&uniqifier=1)
*   Segment zum Löschen vorhandener Tabellen in der DWH-Datenbank eingefügt, damit diese vor Ausführung des Skriptes leer ist (Link: https://colab.research.google.com/drive/1hEBFYWiAFUNKBRinhEY43eOEqXeaHohi#scrollTo=KlUM0-X_KVhk&line=1&uniqifier=1)
*  CREATE Table Statements an neues Datenmodell angepasst
*  Klasse SqlQuery angepasst um Daten aus mehreren verknüpften Tabellen der Quelldatenbank extrahieren zu können
*  SQL Statements zum Extrahieren der benötigten Daten aus den Tabellen der Quelldatenbank ergänzt
* Funktion zum Auslesen der Spaltennamen einer Datenbanktabelle ergänzt



In [ ]:
# Vorsichtshalber: Löschen aller Variablen
%reset -f

# Laden der benötigten Libraries
from google.colab import drive
import sqlite3 as sq
from sqlite3 import Error
import pandas as pd

Mounten des Google Drives, überprüfen des Verzeichnisses, in welchem die Datenbanken liegen.
Außerdem setzen von Pfadangaben, die im späteren Code benötigt werden:


1.   Festlegen des Pfads zur Quelldatenbank (DB_SOURCE)
2.   Festlegen des Pfads zur DWH-Datenbank (DB_DWH)


In [ ]:
# Google Drive mounten, force_remount auf True setzen, damit ein Remount erzwungen wird
drive.mount('/content/gdrive/', force_remount=True)

# Datenbankordner auf dem Shareddrive checken, es müssen source_breast_cancer.db und DWH_breast_cancer.db vorhanden sein
!ls "/content/gdrive/Shareddrives/DMA_Datenprojekt_TeamA/Daten/Datenbank"

# Patiententyp festlegen
patient_type = "breast_cancer"

# Pfad zur Quelldatenbank setzen
DB_SOURCE_PATH = "/content/gdrive/Shareddrives/DMA_Datenprojekt_TeamA/Daten/Datenbank/source_breast_cancer.db"

# Pfad zur DWH-Datenbank setzen
DB_DWH_PATH = "/content/gdrive/Shareddrives/DMA_Datenprojekt_TeamA/Daten/Datenbank/DWH_breast_cancer.db"

# Check
print("\n" + DB_SOURCE_PATH)

print("\n" + DB_DWH_PATH)

# Eventuell vorhandene Tabellen in der DWH-DB löschen
Es muss überprüft werden, ob in der DWH-DB schon Tabellen vorhanden sind, da dieses Skript die Tabellen in der DWH-DB anlegt. Wenn Tabellen vorhanden sind, sollen ausgegeben werden, um welche Tabellen es sich handelt und die Tabellen im Anschluss gelöscht werden.

In [ ]:
# Datenbankverbindung zum DWH aufbauen
conn = sq.connect(DB_DWH_PATH) 
if conn is not None:
  cursor = conn.cursor()
else:
  print("Verbindung fehlgeschlagen. Bitte überprüfen!")

# Alle Tabellennamen aus der Datenbank ziehen
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tablelist = cursor.fetchall()

# Tabellennamen ausgeben, wenn vorhanden
if tablelist == []:
  print("In der Data Warehouse Datenbank sind keine Tabellen vorhanden. Sie können mit der Ausführung des Skriptes fortfahren.")
else:
  print("Folgende Tabellen sind in der Datenbank vorhanden: ")
  print(tablelist)


for tablename in tablelist:
  try:
    # Name der Tabelle in einen String konvertieren, damit String-Operationen durchgeführt werden können
    str_tablename = str(tablename)
    # Entfernen von "('" und "',)" - Übriggebliebene Formatierungs-Elemente aus der Liste
    str_tablename = str_tablename.replace("('", "").replace("',)", "")
    
    # Variable mit dem SQL-Statement erstellen
    droptable = ("DROP TABLE " + str_tablename + ";")
    cursor.execute(droptable)
    print("Tabelle " + str_tablename + " wurde aus der Datenbank gelöscht.")

  except:
    print("In der Datenbank sind keine Tabellen vorhanden. Sie können mit der Ausführung des Skriptes fortfahren.")

# Tabellen in der DB überprüfen, ob richtig gelsöcht wurde
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tablecheck = cursor.fetchall()
print(tablecheck)

# Commit und Close
conn.commit()
conn.close()

# Versionen der verwendeten Pakete abfragen 

Die Versionen der verwendeten Python-Installation und der Python-Pakete abfragen. 

In [ ]:
# Python-Version
print("Python-Version:")
!python --version


# Pandas-Version
print("\n" + "Pandas-Version:")
print("Pandas " + pd.__version__)

# sqlitee-Version
print("\n" + "sqlite3-Version:")
print("sqlite3 " + sq.sqlite_version)


# Erstellung der DWH-Datenbank (Reporting-DB)

In [ ]:
class DB(object):
  #__init__ Funktion, wird aufgerufen, wenn Klasse initiiert wird
  def __init__(self, db_file):
    # Verbindung zur Datenbank herstellen
    self.conn = sq.connect(db_file)
    # Cursor für Operationen auf die Datenbank herstellen
    self.cur = self.conn.cursor()
    self.__init_db()
  
  # Funktion für den Commit der Änderungen auf die Datenbank und das Schließen der Verbindung
  def __del__(self):
      self.conn.commit()
      self.conn.close()

  # Funktion für die Initialisierung der Datenbank
  def __init_db(self):
    # Im Folgenden werden die SQL-Queries angelegt, mithilfe welcher die Tabellen in der DWH-Datenbank erstellt werden
    # Tabellen mit Tabellen mit Präfix "F" = Faktentabellen, Präfix "D" = Dimensionstabellen

    # sql query zum Erstellen der D_patients - Tabelle
    create_D_patients = """CREATE TABLE IF NOT EXISTS D_patients (
                        Id VARCHAR PRIMARY KEY,
                        BIRTHDATE DATE,
                        DEATHDATE DATE,
                        FIRST VARCHAR,
                        LAST VARCHAR,
                        RACE VARCHAR,
                        ETHNICITY VARCHAR,
                        GENDER CHAR(1),
                        HEALTHCARE_EXPENSES FLOAT,
                        HEALTHCARE_COVERAGE FLOAT
                        );"""

    # sql query zum Erstellen der D_payers - Tabelle
    create_D_payers = """CREATE TABLE IF NOT EXISTS D_payers (
                      Id VARCHAR PRIMARY KEY,
                      NAME VARCHAR
                      );"""

    # sql query zum Erstellen der D_snomedct - Tabelle
    create_D_snomedct = """CREATE TABLE IF NOT EXISTS D_snomedct (
                        CODE INTEGER PRIMARY KEY,
                        TERM VARCHAR
                        );"""

    # sql query zum Erstellen der D_rxnorm - Tabelle
    create_D_rxnorm = """CREATE TABLE IF NOT EXISTS D_rxnorm (
                      CODE INTEGER PRIMARY KEY,
                      NAME VARCHAR
                      );"""

    # sql query zum Erstellen der F_encounter_costs - Tabelle
    create_F_encounter_costs = """CREATE TABLE IF NOT EXISTS F_encounter_costs (
                               Id VARCHAR PRIMARY KEY,
                               ENC_START DATETIME,
                               ENC_STOP DATETIME,
                               PATIENT VARCHAR,
                               ENC_PAYER VARCHAR,
                               ENC_BASE_COST FLOAT,
                               ENC_TOTAL_CLAIM_COST FLOAT,
                               ENC_PAYER_COVERAGE FLOAT,
                               ENC_REASON INTEGER,
                               PRO_DATE DATETIME,
                               PRO_CODE INTEGER,
                               PRO_BASE_COST FLOAT,
                               PRO_REASON INTEGER,
                               MED_START DATETIME,
                               MED_STOP DATETIME,
                               MED_CODE INTEGER,
                               MED_BASE_COST FLOAT,
                               MED_DISPENSES INTEGER,
                               MED_TOTAL_COST FLOAT,
                               MED_PAYER_COVERAGE FLOAT,
                               MED_PAYER VARCHAR,
                               MED_REASON INTEGER,
                               CON_START DATETIME,
                               CON_STOP DATETIME,
                               CON_CODE INTEGER,
                               FOREIGN KEY (PATIENT)
                                REFERENCES D_patients (Id),
                               FOREIGN KEY
                                (ENC_PAYER) REFERENCES D_payers (Id),
                               FOREIGN KEY
                                (MED_PAYER) REFERENCES D_payers (Id),
                               FOREIGN KEY
                                (ENC_REASON) REFERENCES D_snomedct (Code),
                               FOREIGN KEY (PRO_CODE)
                                REFERENCES D_snomedct (Code),
                               FOREIGN KEY (PRO_REASON)
                                REFERENCES D_snomedct (Code),
                               FOREIGN KEY (MED_REASON)
                                REFERENCES D_snomedct (Code),
                               FOREIGN KEY (CON_CODE)
                                REFERENCES D_snomedct (Code),
                               FOREIGN KEY (MED_CODE)
                                REFERENCES D_rxnorm (Code)
                               );"""

    # CREATE-TABLE-Statements in eine Liste zusammenfassen
    create_tables = [create_D_patients, # Daten zu Patienten
                     create_D_payers, # Daten zu Versicherungen
                     create_D_snomedct, # SNOMED CT Referenztabelle
                     create_D_rxnorm, # RXNorm Referenztabelle
                     create_F_encounter_costs, # Daten zum Behandlungsfall (zentrale Faktentabelle)
                     ]
     
    if self.conn is not None:
      for query in create_tables:
          # Führ jedes SQL-Statement aus der create_tables Liste aus
          self.cur.execute(query)
          print('Table created')
    else:
      print('Connection to database failed')



#ETL-Strecke (Extract-Transform-Load) zur Befüllung der DWH-Datenbank

In [ ]:
# Klasse um SQL-Queries zu exportieren
class SqlQuery:
  #__init__ Funktion, wird aufgerufen, wenn Klasse initiiert wird

  """ sink_table = Zieltabelle in der DWH-Datenbank
      sink_column_names = Spaltennamen in der DWH-Datenbank
      source_extract = SQL Query um Daten aus der Quelldatenbank zu extrahieren
  """
  def __init__(self, source_query, sink_table, sink_column_names):
    self.source_query = source_query
    self.sink_table = sink_table
    # Anzahl der column_names für loop
    self.sink_column_numbers = len(sink_column_names)
    # Spaltennamen der Quelltabellen als Liste, kommasepariert
    self.sink_column_names = ', '.join(sink_column_names)

  # Funktion extract_query gibt ein SQL-SELECT-Statement auf die Spalten der Quelltabelle in der Quelldatenbank source_breast_cancer.db zurück
  def extract_query(self):
    return self.source_query

  # Funktion load_query gibt ein SQL INSERT-INTO-Statement zurück, welches die Werte in die Tabellen der DWH-Datenbank überträgt
  def load_query(self):
    values_str = '?,' * self.sink_column_numbers
    # print("*****", values_str, column_names, column_numbers)
    values_str = values_str[:-1]
    return 'INSERT OR REPLACE INTO ' + self.sink_table + ' VALUES (' + values_str + ')'

    # return 'INSERT INTO ' + self.sink_table + '(' + self.column_names + ') VALUES (' + values_str + ')'
  


In [ ]:
# Funktion etl extrahiert die Daten aus der Quelldatenbank und lädt die Daten in die DWH-Datenbank
def etl(query, source_cnx, target_cnx):

  """source_cnx = Verbindung zur Quelldatenbank
     target_cnx = Verbindung zur DWH-Datenbank
     query = Objekt mit der Klasse "SqlQuery"
  """
  source_cursor = source_cnx.cursor()
  # extract_query() ist Methode aus der Klasse SqlQuery
  source_cursor.execute(query.extract_query())
  data = source_cursor.fetchall()
  source_cursor.close()

  # Wenn Daten in die Variable "data" über fetchall() gezogen wurden, wird die folgende Schleife ausgeführt 
  if data:
    target_cursor = target_cnx.cursor()
    # Multi-Query, .load_query() ist Methode aus der Klasse "SqlQuery"
    target_cursor.executemany(query.load_query(), data)
    print('Daten wurden erfolgreich in die DWH-Datenbank übertragen!')
    # Änderungen commiten
    target_cnx.commit()
    # Verbindung zur DWH_DB schließen
    target_cursor.close()
  else:
    print('Es wurden keine Daten übertragen.')

# Funktion etl_process durchläuft die Liste der SQL-Queries und ruft jeweils die Funktion etl auf
def etl_process(queries, target_cnx, db_source):
  """ queries: query-Liste mit SQL
      target_cnx: Verbindung zur DWH-Datenbank 
      db_source: Pfad zur Quelldatenbank
  """  
  # Verbindung zur Quelldatenbank herstellen
  try:
    source_cnx = sq.connect(db_source)
  except Error as err:
    print(err)
  
  # Durch alle queries Loopen, die als Liste übergeben wurden
  for query in etl_queue:
    # Funktion etl() aufrufen, welche die Daten aus den Tabellen der Quelldatenbank extrahiert und in die DWH-Datenbank lädt
    etl(query, source_cnx, target_cnx)
    
  # Verbindung zur Quelldatenbank schließen
  source_cnx.close()


In [ ]:
# Funktion um alle Spaltennamen einer Tabelle tb in einer Datenbank db als Liste auszugeben
def get_column_names(db, tb):
  tb_colnames = []

  try:
    db_cnx = sq.connect(db)
  except Error as err:
    print(err)

  db_cur = db_cnx.cursor()
  db_cur.execute('SELECT name FROM pragma_table_info("'+ tb + '");')

  for column in db_cur.fetchall():
    tb_colnames.append(column[0])

  return tb_colnames

In [ ]:
# Objekt der Klasse DB erzeugen, als Argument den Pfad zur DWH-Datenbank übergeben
dwh_db = DB(DB_DWH_PATH)

In [ ]:
print('ETL-Prozess wird gestartet.')   
# Liste für die SqlQueries anlegen
etl_queue = []


# Im Folgenden werden Objekte mit Klasse SqlQuery erzeugt
""" Übergeben werden:
    a = Quelltabelle aus der Quelldatenbank
    b = Zieltabelle in der DWH-Datenbank
    c = Spaltennamen in der Zieltabelle in der DWH-Datenbank
"""

# Daten zu Patienten
D_patients_columns = get_column_names(DB_DWH_PATH, 'D_patients')
D_patients_extract = """
                      SELECT id,
                             birthdate,
                             deathdate,
                             first,
                             last,
                             race,
                             ethnicity,
                             gender,
                             healthcare_expenses,
                             healthcare_coverage
                      FROM   patients;
                     """
sql_query_D_patients = SqlQuery(D_patients_extract, "D_patients", D_patients_columns)
etl_queue.append(sql_query_D_patients)

# Daten zu Versicherungen
D_payers_columns = get_column_names(DB_DWH_PATH, 'D_payers')
D_payers_extract = """
                    SELECT id,
                           name
                    FROM   payers;
                   """
sql_query_D_payers = SqlQuery(D_payers_extract, "D_payers", D_payers_columns)
etl_queue.append(sql_query_D_payers)

# SNOMED CT Codes
D_snomedct_columns = get_column_names(DB_DWH_PATH, 'D_snomedct')
D_snomedct_extract = """
                      SELECT pro.code              AS CODE,
                             pro.description       AS TERM
                      FROM   procedures AS pro
                      WHERE  NOT pro.code = ""
                      UNION
                      SELECT pro.reasoncode        AS CODE,
                             pro.reasondescription AS TERM
                      FROM   procedures AS pro
                      WHERE  NOT pro.reasoncode = ""
                      UNION
                      SELECT enc.reasoncode        AS CODE,
                             enc.reasondescription AS TERM
                      FROM   encounters AS enc
                      WHERE  NOT enc.reasoncode = ""
                      UNION
                      SELECT med.reasoncode        AS CODE,
                             med.reasondescription AS TERM
                      FROM   medications AS med
                      WHERE  NOT med.reasoncode = ""
                      UNION
                      SELECT con.code        AS CODE,
                             con.description AS TERM
                      FROM   conditions AS con
                      WHERE  NOT con.code = "";
                    """
sql_query_D_snomedct = SqlQuery(D_snomedct_extract, "D_snomedct", D_snomedct_columns)
etl_queue.append(sql_query_D_snomedct)

# RXNorm Codes
D_rxnorm_columns = get_column_names(DB_DWH_PATH, 'D_rxnorm')
D_rxnorm_extract = """
                      SELECT DISTINCT(code),
                             description
                      FROM   medications;
                    """
sql_query_D_rxnorm = SqlQuery(D_rxnorm_extract, "D_rxnorm", D_rxnorm_columns)
etl_queue.append(sql_query_D_rxnorm)

# Daten zu Behandlungsfällen und Kosten
F_encounter_costs_columns = get_column_names(DB_DWH_PATH, 'F_encounter_costs')
F_encounter_costs_extract = """
                              SELECT enc.id,
                                     enc.start,
                                     enc.stop,
                                     enc.patient,
                                     enc.payer,
                                     enc.base_encounter_cost,
                                     enc.total_claim_cost,
                                     enc.payer_coverage,
                                     enc.reasoncode,
                                     pro.date,
                                     pro.code,
                                     pro.base_cost,
                                     pro.reasoncode,
                                     med.start,
                                     med.stop,
                                     med.code,
                                     med.base_cost,
                                     med.dispenses,
                                     med.total_cost,
                                     med.payer_coverage,
                                     med.payer,
                                     med.reasoncode,
                                     con.start,
                                     con.stop,
                                     con.code
                              FROM   encounters AS enc
                                     LEFT JOIN procedures AS pro
                                            ON enc.id = pro.encounter
                                     LEFT JOIN medications AS med
                                            ON enc.id = med.encounter
                                     LEFT JOIN conditions AS con
                                            ON enc.id = con.encounter;
                            """
sql_query_F_encounter_costs = SqlQuery(F_encounter_costs_extract, "F_encounter_costs", F_encounter_costs_columns)
etl_queue.append(sql_query_F_encounter_costs)

# list for iteration
etl_queue

In [ ]:
# Verbindung zur DWH-Datenbank herstellen
target_cnx = dwh_db.conn
etl_process(etl_queue, target_cnx, DB_SOURCE_PATH)

In [ ]:
# Änderungen Committen
target_cnx.commit()

In [ ]:
# Erstellte Tabellen checken
dwh_cursor = target_cnx.cursor()
dwh_cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(dwh_cursor.fetchall())


In [ ]:
# Spalten einer Tabelle stichprobenartig checken
dwh_cursor.execute('PRAGMA table_info(' + "F_encounter_costs" + ');')
dwh_cursor.fetchall()

In [ ]:
# Testen einer Selektion: F_encounter_costs
dwh_cursor.execute("SELECT * from F_encounter_costs")
rows = dwh_cursor.fetchall()
for row in rows[:10]:
  print(row)

In [ ]:
# Testen einer Selektion: D_patients
dwh_cursor.execute("SELECT * from D_patients")
rows = dwh_cursor.fetchall()
for row in rows[:10]:
  print(row)

In [ ]:
# Testen einer Selektion: D_snomedct
dwh_cursor.execute("SELECT * from D_snomedct")
rows = dwh_cursor.fetchall()
for row in rows[:10]:
  print(row)

In [ ]:
# Testen einer Selektion: D_rxnorm
dwh_cursor.execute("SELECT * from D_rxnorm")
rows = dwh_cursor.fetchall()
for row in rows[:10]:
  print(row)

In [ ]:
# Testen einer Selektion: D_payers
dwh_cursor.execute("SELECT * from D_payers")
rows = dwh_cursor.fetchall()
for row in rows[:10]:
  print(row)

In [ ]:
# Verbindung schließen
target_cnx.close()